In [1]:
import numpy as np
import pandas as pd
from tools.get_data.get_data_h5 import *
from pandasgui import show
import talib as ta
import datetime
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import roll_time_series
import numpy as np
import pandas as pd
import os
import plotly.express as px
from IPython.display import display



### 1. 获取factor数据

In [4]:
from importlib import reload
import factors as dl
reload(dl)
run = 0
if run == 1:
    data = get_data(symbol="rb99", freq="minbar", type="future",
                    source_dir="Y:/DataBase_RQ_files/")
    data["sma_diff_5"] = dl.sma_diff(data, 5)
    data["sma_diff_20000"] = dl.sma_diff(data, 20000)
    data["ema_diff_300"] = dl.ema_diff(data, 300)
    data["ema_diff_1000"] = dl.ema_diff(data, 1000)
    data["sma_of_sma_5_10"] = dl.sma_of_sma(data, 5, 10)
    data["ema_of_ema_10_30"] = dl.ema_of_ema(data, 10, 30)
    data["sma_diff_sma_5_100"] = dl.sma_diff_sma(data, 5, 100)
    data["high_60"] = dl.high(data, 60)
    data["low_60"] = dl.low(data, 60)
    data["high_60"] = dl.high(data, 60)
    data["low_60"] = dl.low(data, 60)
    data["wl_60"] = dl.wl(data, 60)
    data["wl_60"] = dl.wl(data, 60)
    data["macd_dif"], data["macd_dea"], data["macd_hist"], data["macd_signal"] = dl.MACD(
        data, fast=10, slow=60, mid=15)
    data["slope"] = dl.slope(data, 60)
    data["rsi_60"] = dl.rsi(data, 60)
    data["sar"] = dl.sar(data)
    data["rwr_60"] = dl.rwr(data, 60)
    data["rsi_100"] = dl.rsi(data, 100)
    data["rwr_100"] = dl.rwr(data, 100)
    data["aroon_up"], data["aroon_down"], data["aroon"] = dl.aroon(data, 60)
    data["tendstrength"] = dl.tendstrength(data, 120)
    data["boll"] = dl.boll(data, 200)
    data["don"] = dl.don(data, 100)
    data["sf01"] = dl.sf01(data, 60)
    data["cor_vol"] = dl.cor_vol(data, 200)
    data["cor_oi"] = dl.cor_oi(data, 1000)

    l = [1, 3, 5] + (list(range(10, 201, 5)))
    for i in l:
        data["shift_{}_rtn".format(i)] = dl.rtn_shift(data.close, -i)
    print("liqka前工作完成", len(data))

    data["long_liqka"] = dl.long_liqka(data)
    data["short_liqka"] = dl.short_liqka(data)
    data.dropna(inplace=True)
    data.to_parquet(".//data//factors.parquet")
    

data = pd.read_parquet(".//data//factors.parquet")
# show(data.head(50))

### 2. 因子分析

#### 2.1 创建因子rank_df/results_df(耗时长)

In [ ]:
# rank_df = pd.read_parquet(".//data//symbol_30000_20_rank_df.parquet")
# rank_df2 = rank_df[-100000:]
# results_df = pd.read_parquet(".//data//symbol_30000_20_results_df.parquet")
# rank_df[[col for col in rank_df.columns if "rank" in col]].head(50)


In [ ]:
from importlib import reload    
import factor_analysis as fa
reload(fa)
data = pd.read_parquet(".//data//factors.parquet")
data.set_index("datetime", inplace=True, drop=True)
factors_cols = []
rtn_cols = []
for col in data.columns:
    if col not in ['datetime', 'trading_date', "symbol"]:
        if "rtn" not in col and "liqka" not in col:
            factors_cols.append(col)
        else:
            rtn_cols.append(col)
factors = data[factors_cols]
rtn = data[rtn_cols]
run = 0
if run == 1:
    fal = fa.FactorAnalysis_ori()
    rank_df, results_df = fal.factor_ranked(
    factors, rtn, save=True, sample_size=60000, bins=20)

    rank_df, results_df = fal.factor_ranked(factors, rtn,save=True,sample_size=30000,bins=20)
    rank_df, results_df = fal.factor_ranked(factors, rtn,save=True,sample_size=20000,bins=15)
    rank_df, results_df = fal.factor_ranked(factors, rtn,save=True,sample_size=10000,bins=15)
    rank_df, results_df = fal.factor_ranked(factors, rtn,save=True,sample_size=5000,bins=10)
    rank_df, results_df = fal.factor_ranked(factors, rtn,save=True,sample_size=2000,bins=10)

#### 2.2 因子分析

In [ ]:
#因子分析, 画出最佳因子的累计收益图
from importlib import reload
import factor_analysis as fa
reload(fa)
fal = fa.FactorAnalysis_ori()
seletced ={} #选择的因子
show_up = 6
for  i in os.listdir(".//data//"):
    if "_results_df" in i:
        _rank = i.replace("_results_df", "_rank_df")
        _rank_df = pd.read_parquet(".//data//"+_rank)
        sample_size,bins = i[7:-19].split("_")
        sample_size =int(sample_size)
        bins = int(bins)
        # _results_df = fal.cal_rank_results(factors, rtn, rank_df = _rank_df, sample_size=sample_size, bins=bins, save=True)
        # _results_df.drop(['long_liqka_mean', 'short_liqka_mean','long_liqka_win_rate', 'short_liqka_win_rate'],axis=1,inplace=True)
        
        _results_df = pd.read_parquet(".//data//"+i)
        seletced[i[7:-19]] = fal.factors_select(_results_df, win_rate=0, rtn=-3,count=None,sorted="win_rate")
        display(f"seletced: { i[7:-19]}" , seletced[i[7:-19]].head())
        fal.cumsum_plot(rank_df = _rank_df, 
                        results_df = _results_df,
                        n =3,
                        sorted="mean_rtn")
        
        show_up -= 1
        if show_up == 0:
            break


### 3. 因子有效期分析


In [ ]:
seletced["10000_15"]
# _results_df 
print("sample_size and bins", i[7:-19])

In [ ]:
rank_df = pd.read_parquet(".//data//symbol_30000_20_rank_df.parquet")
rank_df

In [ ]:
seletced["10000_15"]

In [ ]:

"short_liqka_mean"[:-5]


In [ ]:
_results_df

In [ ]:
rank_df[['sar_rank', 'rsi_60_rank', 'rwr_60_rank', 'rsi_100_rank','shift_155_rtn', 'shift_160_rtn', 'shift_165_rtn', 'shift_170_rtn']].head()